Import required packages

In [ ]:
import numpy as np
import tkinter
import matplotlib.pyplot as plt
import keras
from keras.datasets import mnist,cifar10,cifar100,fashion_mnist
from keras.models import Sequential,Model
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Input,Reshape,concatenate
from keras import optimizers
from sklearn.metrics import precision_score,recall_score,f1_score
import keras.backend as K
from scipy.io import loadmat
import os
import h5py

Intialize some global variables

In [8]:
batchsize = 64
epoch = 5
num_classes1 = 10
num_classes2 = 26
lr = 0.0001

Function to load the inputs

In [19]:
def load_input(x):
    if x == 'mnist':
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
        return (x_train, y_train), (x_test, y_test)
    else:
        if x == 'cifar10':
            (x_train, y_train), (x_test, y_test) = cifar10.load_data()
            return (x_train, y_train), (x_test, y_test)

        else:
            if x == 'cifar100':
                (x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='coarse')
                return (x_train, y_train), (x_test, y_test)
            else:
                if x == 'fashion_mnist':
                    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
                    return (x_train, y_train), (x_test, y_test)
                else:
                    if x == 'svhn':
                        x_train = loadmat('/home/nagarjun/datasets/svhn_cnn/x_train.mat')
                        x_test = loadmat('/home/nagarjun/datasets/svhn_cnn/x_test.mat')
                        y_train = loadmat('/home/nagarjun/datasets/svhn_cnn/y_train.mat')
                        y_test = loadmat('/home/nagarjun/datasets/svhn_cnn/y_test.mat')
                        x_train = x_train['x_train']
                        x_test = x_test['x_test']
                        y_train = y_train['y_train']
                        y_test = y_test['y_test']

                        return (x_train, y_train-1), (x_test, y_test-1)

                    else:
                        if x == 'nist_char':
                            x_train = loadmat('/home/nagarjun/datasets/nist_char_cnn/x_train.mat')
                            x_test = loadmat('/home/nagarjun/datasets/nist_char_cnn/x_test.mat')
                            y_train = loadmat('/home/nagarjun/datasets/nist_char_cnn/y_train.mat')
                            y_test = loadmat('/home/nagarjun/datasets/nist_char_cnn/y_test.mat')
                            x_train = x_train['x_train']
                            x_test = x_test['x_test']
                            y_train = y_train['y_train']
                            y_test = y_test['y_test']
                            return (x_train, y_train - 1), (x_test, y_test - 1)

Declare the 1D LeNet 

In [14]:
def lenet_CNN(x):

    if x == 'mnist' or x == 'fashion_mnist' or x == 'nist_char':

        input = Input(shape=(x_train.shape[1],))

        conv1 = Dense(3456,activation='relu')(input)
        conv1 = Reshape(target_shape=(24, 24, 6))(conv1)

        pool1 = MaxPooling2D((2,2))(conv1)
        pool1 = Reshape(target_shape=(12*12*6,))(pool1)

        conv2 = Dense(1024,activation='relu')(pool1)
        conv2 = Reshape(target_shape=(8,8,16))(conv2)
        pool2 = MaxPooling2D((2,2))(conv2)
        pool2 = Flatten()(pool2)

        dense1 = Dense(120,activation='relu')(pool2)
        dense2 = Dense(84,activation='relu')(dense1)
        if x == 'mnist':
            out = Dense(num_classes1,activation='softmax')(dense2)
        else:
            if x == 'fashion_mnist':
                out = Dense(num_classes1, activation='softmax')(dense2)
            else:
                out = Dense(num_classes2, activation='softmax')(dense2)

        model = Model(input,out)
        return model

    else:
        if x == 'cifar10' or x =='svhn':

            in1 = Input(shape=(x_train1.shape[1],))

            out1 = Dense(4704, activation='relu')(in1)

            in2 = Input(shape= (x_train2.shape[1],))
            out2 = Dense(4704, activation='relu')(in2)

            in3 = Input(shape= (x_train3.shape[1],))
            out3 = Dense(4704, activation='relu')(in3)

            #input = concatenate([in1, in2, in3])

            conv1 = concatenate([out1, out2, out3])

            conv1 = Reshape(target_shape=(28, 28, 18))(conv1)

            pool1 = MaxPooling2D((2, 2))(conv1)
            pool1 = Reshape(target_shape=(14 * 14 * 18,))(pool1)

            conv2 = Dense(1600, activation='relu')(pool1)
            conv2 = Reshape(target_shape=(10, 10, 16))(conv2)
            pool2 = MaxPooling2D((2, 2))(conv2)
            pool2 = Flatten()(pool2)

            dense1 = Dense(120, activation='relu')(pool2)
            dense2 = Dense(84, activation='relu')(dense1)
            out = Dense(num_classes1, activation='softmax')(dense2)

            model = Model([in1,in2,in3], out)
            return model

Function to save trained model

In [ ]:
def save_trained_model(x):
    
    if x == 'mnist':
        # serialize model to JSON
        model_json1 = model.to_json()
        with open("model.json1", "w") as json_file:
            json_file.write(model_json1)
        # serialize weights to HDF5
        model.save_weights('/home/nagarjun/datasets/trained_models/mnist_cnn_model.h5')
        print("Saved model to disk")

    else:
        if x == 'fashion_mnist':
            # serialize model to JSON
            model_json2 = model.to_json()
            with open("model.json2", "w") as json_file:
                json_file.write(model_json2)
            # serialize weights to HDF5
            model.save_weights('os.path.join(path,'trained_models',fashion_mnist_cnn_model.h5'))
            print("Saved model to disk")
        else:
            if x == 'nist_char':
                # serialize model to JSON
                model_json3 = model.to_json()
                with open("model.json3", "w") as json_file:
                    json_file.write(model_json3)
                # serialize weights to HDF5
                model.save_weights(os.path.join(path,'trained_models','nist_cnn_model.h5'))
                print("Saved model to disk")
            else:
                if x == 'cifar10':
                    # serialize model to JSON
                    model_json4 = model.to_json()
                    with open("model.jsoncifar", "w") as json_file:
                        json_file.write(model_json4)
                    # serialize weights to HDF5
                    model.save_weights(os.path.join(path,'trained_models','cifar10_cnn_model.h5'))
                    print("Saved model to disk")
                else:
                    if x == 'svhn':
                        # serialize model to JSON
                        model_json5 = model.to_json()
                        with open("model.jsonsvhn", "w") as json_file:
                            json_file.write(model_json5)
                        # serialize weights to HDF5
                        model.save_weights(os.path.join(path,'trained_models','svhn_cnn_model.h5'))
                        print("Saved model to disk")

1. Load model + inputs 
2. Compile model 
3. Fit the model
4. Evaluate model with metrics Top-1 accuracy, Top-5, Precision, Recall and F1-score

In [20]:
x = ['mnist','fashion_mnist','nist_char','cifar10','svhn']

for j in range(len(x)):

    (x_train, y_train), (x_test, y_test) = load_input(x[j])

    x_train = x_train/255
    x_test = x_test/255

    if x[j] == 'mnist' or x[j] == 'fashion_mnist':

        y_train = keras.utils.to_categorical(y_train, num_classes=num_classes1)
        y_test = keras.utils.to_categorical(y_test, num_classes=num_classes1)

        [num_images_train,m,n] = np.shape(x_train)
        [num_images_test,_,_]  = np.shape(x_test)

        x_train = np.reshape(x_train,[num_images_train,m,n,1])
        x_test = np.reshape(x_test,[num_images_test,m,n,1])

        [num_images_train, m, n,channels] = np.shape(x_train)
    else:
        if x[j] == 'cifar10' or x[j] == 'svhn':
            y_train = keras.utils.to_categorical(y_train, num_classes=num_classes1)
            y_test = keras.utils.to_categorical(y_test, num_classes=num_classes1)

            x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1] * x_train.shape[2] * x_train.shape[3]))
            x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1] * x_test.shape[2] * x_test.shape[3]))

            print(x_train.shape)

            # [num_images_train, m, n,channels] = np.shape(x_train)
            # [num_images_test, _, _,_] = np.shape(x_test)

        else:
            if x[j] == 'nist_char':

                y_train = keras.utils.to_categorical(y_train, num_classes=num_classes2)
                y_test = keras.utils.to_categorical(y_test, num_classes=num_classes2)

                [num_images_train, m, n] = np.shape(x_train)
                [num_images_test,_,_] = np.shape(x_test)

                x_train = np.reshape(x_train, [num_images_train, m, n, 1])
                x_test = np.reshape(x_test, [num_images_test, m, n, 1])

                [num_images_train, m, n, channels] = np.shape(x_train)

    model = select_model(x[j])
    print(model.summary())

    model.compile(optimizer=optimizers.Adam(lr),loss='categorical_crossentropy',metrics=['accuracy','top_k_categorical_accuracy'])

    model.fit(x_train,y_train,batch_size=batchsize,epochs=epoch,verbose=0)

    [loss_test,score,top5] = model.evaluate(x_test,y_test,batch_size=batchsize,verbose=0)
    print(x[j],'\n')

    print('Test Accuracy\n',score,top5)

    y_pred_test = model.predict(x_test, batch_size=batchsize)

    precision_test = precision_score(y_test, y_pred_test.round(), average='micro')
    recall_test = recall_score(y_test, y_pred_test.round(), average='micro')

    #f1_train = f1_score(y_train, y_pred_train.round(), average='micro')
    f1_test = f1_score(y_test, y_pred_test.round(), average='micro')

    print('Precision-test:\n', precision_test)
    print('Recall-test:\n', recall_test)

    print('F1-score-test:\n', f1_test)

    save_trained_model(x[j])

    K.clear_session()

NameError: global name 'cifar10' is not defined